In [2]:
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
from prophet import Prophet
import seaborn as sns
import os
sns.set()

Importing plotly failed. Interactive plots will not work.


In [16]:

# Cargar el conjunto de datos en un DataFrame
df = pd.read_csv("txt_eda/unpopulation_dataportal_20230223061750.csv")

# Agrupar los datos por el valor de la columna "location"
grupos = df.groupby("location")

for pais, datos_pais in grupos:
    # Crear el directorio para el país si no existe
    directorio_pais = f"{pais}"
    if not os.path.exists(directorio_pais):
        os.makedirs(directorio_pais)

# Iterar sobre los grupos y dividirlos en subconjuntos separados para cada valor único en la columna "indicatorshortname"
for grupo_nombre, grupo_datos in grupos:
    for indicador_nombre, indicador_datos in grupo_datos.groupby("indicatorshortname"):
        # Guardar el subconjunto como un archivo CSV separado
        archivo_nombre = f"{grupo_nombre}_{indicador_nombre}.csv"
        indicador_datos.to_csv(archivo_nombre, index=False)

In [19]:
# Crear directorio para los gráficos si no existe
graphs_dir = "txt_eda/paises/graphs/proph"
if not os.path.exists(graphs_dir):
    os.makedirs(graphs_dir)


# Obtener lista de nombres de archivos
file_names = os.listdir("txt_eda/paises/0 aa crude rate net total migrnt")

# Iterar sobre los nombres de archivo y crear gráficos
for file_name in file_names:
    if file_name.endswith(".csv"):
        path = os.path.join("txt_eda/paises/0 aa crude rate net total migrnt", file_name)
        df = pd.read_csv(path)

        # Crear un gráfico de línea para el DataFrame actual
        fig, axs = plt.subplots(2, figsize=(10, 10), facecolor='w')
        elemento = df["indicatorshortname"].iloc[0]
        axs[0].plot(df["time"], df["value"])
        axs[0].set_xlabel("Fecha", fontsize=14)
        axs[0].set_ylabel("Valor", fontsize=14)
        axs[0].set_title(elemento, fontsize=16)
        axs[0].tick_params(axis='both', which='major', labelsize=12)

        # Crear un gráfico de dispersión para el DataFrame actual
        axs[1].scatter(df["time"], df["value"])
        axs[1].set_xlabel("Fecha", fontsize=14)
        axs[1].set_ylabel("Valor", fontsize=14)
        axs[1].set_title(elemento, fontsize=16)
        axs[1].tick_params(axis='both', which='major', labelsize=12)

        # Ajustar los espacios entre los gráficos
        fig.tight_layout()

        # Cambiar caracteres no permitidos en el nombre del archivo por guiones bajos
        file_name = file_name.replace(":", "_").replace("/", "_").replace("\\", "_").replace("*", "_").replace("?", "_").replace("\"", "_").replace("<", "_").replace(">", "_").replace("|", "_")

        # Guardar el gráfico
        graph_path = os.path.join(graphs_dir, file_name.replace(".csv", ".png"))
        plt.savefig(graph_path)

        # Cerrar la figura
        plt.close(fig)



In [5]:
dir_path = "txt_eda/paises/00paises/favorable"

# Obtener lista de nombres de archivos
file_names = os.listdir(dir_path)

# Iterar sobre los nombres de archivo y cambiar las comas por puntos en la columna "value"
for file_name in file_names:
    if file_name.endswith(".csv"):
        path = os.path.join(dir_path, file_name)
        df = pd.read_csv(path)

        # Eliminar valores nulos o vacíos en la columna "value"
        df = df.dropna(subset=["value"])

        # Convertir la columna "value" a string
        df["value"] = df["value"].astype(str)

        # Reemplazar comas por puntos en la columna "value"
        df["value"] = df["value"].str.replace(",", ".")

        # Guardar el dataframe modificado en el mismo archivo
        df.to_csv(path, index=False)

In [7]:


# Directorios con los datos
favorable_dir = "txt_eda/paises/00paises/favorable"
desfavorable_dir = "txt_eda/paises/00paises/desfavorable"

# Función para leer y formatear los datos
def read_data(file_path):
    df = pd.read_csv(file_path)
    df = df[["time", "value"]]
    df = df.rename(columns={"time": "ds", "value": "y"})
    df["ds"] = pd.to_datetime(df["ds"], format="%Y")
    return df

# Función para crear y guardar el gráfico
def plot_and_save(df, title, file_name):
    m = Prophet()
    m.fit(df)
    future = m.make_future_dataframe(periods=30, freq='Y')
    forecast = m.predict(future)
    fig = m.plot(forecast)
    plt.title(title)
    plt.xlabel("Año")
    plt.ylabel("Valor")
    plt.savefig(file_name)
    plt.close(fig)

# Crear directorios para los gráficos si no existen
graphs_dir_fav = "txt_eda/paises/00paises/favorable/graphs"
graphs_dir_desfav = "txt_eda/paises/00paises/desfavorable/graphs"
for dir_path in [graphs_dir_fav, graphs_dir_desfav]:
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

# Iterar sobre los archivos y crear los gráficos
for file_name in os.listdir(favorable_dir):
    if file_name.endswith(".csv"):
        file_path = os.path.join(favorable_dir, file_name)
        df = read_data(file_path)
        title = file_name.replace(".csv", "")
        graph_path = os.path.join(graphs_dir_fav, file_name.replace(".csv", ".png"))
        plot_and_save(df, title, graph_path)

for file_name in os.listdir(desfavorable_dir):
    if file_name.endswith(".csv"):
        file_path = os.path.join(desfavorable_dir, file_name)
        df = read_data(file_path)
        title = file_name.replace(".csv", "")
        graph_path = os.path.join(graphs_dir_desfav, file_name.replace(".csv", ".png"))
        plot_and_save(df, title, graph_path)


10:49:51 - cmdstanpy - INFO - Chain [1] start processing
10:49:51 - cmdstanpy - INFO - Chain [1] done processing
10:49:51 - cmdstanpy - INFO - Chain [1] start processing
10:49:52 - cmdstanpy - INFO - Chain [1] done processing
10:49:52 - cmdstanpy - INFO - Chain [1] start processing
10:49:52 - cmdstanpy - INFO - Chain [1] done processing
10:49:53 - cmdstanpy - INFO - Chain [1] start processing
10:49:53 - cmdstanpy - INFO - Chain [1] done processing
10:49:53 - cmdstanpy - INFO - Chain [1] start processing
10:49:53 - cmdstanpy - INFO - Chain [1] done processing
10:49:54 - cmdstanpy - INFO - Chain [1] start processing
10:49:54 - cmdstanpy - INFO - Chain [1] done processing
10:49:55 - cmdstanpy - INFO - Chain [1] start processing
10:49:55 - cmdstanpy - INFO - Chain [1] done processing
10:49:55 - cmdstanpy - INFO - Chain [1] start processing
10:49:56 - cmdstanpy - INFO - Chain [1] done processing
10:49:56 - cmdstanpy - INFO - Chain [1] start processing
10:49:56 - cmdstanpy - INFO - Chain [1]